In [49]:
!pip install -q qiskit
!pip install -q pylatexenc
!pip install -q qiskit_aer
%pip install qiskit_ibm_runtime


In [54]:
# Two-Spin Heisenberg Model
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit_aer import AerSimulator
import numpy as np
import scipy.linalg

J = 1.0
t = np.pi/4

X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

H = J*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))
U = Operator(scipy.linalg.expm(-1j*H*t))

qc = QuantumCircuit(2)
qc.h([0,1])
qc.unitary(U, [0,1])
qc.measure_all()

sim = AerSimulator()
result = sim.run(transpile(qc, sim)).result()
print(result.get_counts())




{'11': 257, '01': 275, '10': 236, '00': 256}


In [55]:
# Three-Spin Heisenberg Model
def build_3spin_H(J):
    H = np.zeros((8,8), dtype=complex)
    I = np.eye(2)

    for a, b in [(0,1), (1,2), (2,0)]:
        xx = np.kron(np.kron(X if a==0 else I, X if b==1 else I), I)
        yy = np.kron(np.kron(Y if a==0 else I, Y if b==1 else I), I)
        zz = np.kron(np.kron(Z if a==0 else I, Z if b==1 else I), I)
        if a == 1:
            xx = np.kron(I, np.kron(X,X))
            yy = np.kron(I, np.kron(Y,Y))
            zz = np.kron(I, np.kron(Z,Z))
        if a == 2:
            xx = np.kron(X, np.kron(I,X))
            yy = np.kron(Y, np.kron(I,Y))
            zz = np.kron(Z, np.kron(I,Z))
        H += J*(xx + yy + zz)
    return H

H_3spin = build_3spin_H(1.0)
U_3spin = Operator(scipy.linalg.expm(-1j*H_3spin*t))

qc3 = QuantumCircuit(3)
qc3.h([0,1,2])
qc3.unitary(U_3spin, [0,1,2])
qc3.measure_all()

result3 = sim.run(transpile(qc3, sim)).result()
print(result3.get_counts())

{'101': 107, '001': 135, '000': 127, '010': 134, '100': 127, '110': 140, '111': 121, '011': 133}
